In order to perform training on the EDSR x4 model, we need to:

1. Get the repository of the model, with all the files and modifications of our interest.
2. Have properly located in disk the dataset (or datasets) we are going to use.
3. Have properly located in disk the trained model we want to use, if any.
4. Either set-up a template with arguments for the model script, or provide such arguments in the command line while running the training.
5. Run the training command on a terminal.

We have condensed the configurations from our previous notebooks in this single notebook for a simpler use. We can check each corresponding notebook for more information.

We will use the directories:

In [ ]:
import os

dir_base = os.getcwd()
dir_edsrpytorch = os.path.join(dir_base,"EDSR-PyTorch")     # ./EDSR-PyTorch
dir_src = os.path.join(dir_edsrpytorch,"src")               # ./EDSR-PyTorch/src
dir_pretrain = os.path.join(dir_edsrpytorch, "pre-train")   # ./EDSR-PyTorch/pre-train

# 1. Setting up the repository with the model

We can additionally place the following files inside the src folder of the repository:
* main_use.py: For a simpler application of the main.py script to test a trained model.
* freeze.py: To be able to freeze layers of the model during training.

For more information about the following cells, we can check our previous notebooks.

## 1.1 Getting the repository

In [ ]:
# Cloning the repository as instructed in https://github.com/sanghyun-son/EDSR-PyTorch
!git clone https://github.com/thstkdgus35/EDSR-PyTorch {dir_edsrpytorch}

Cloning into '/content/EDSR-PyTorch'...
remote: Enumerating objects: 806, done.
remote: Total 806 (delta 0), reused 0 (delta 0), pack-reused 806
Receiving objects: 100% (806/806), 63.09 MiB | 22.36 MiB/s, done.
Resolving deltas: 100% (516/516), done.


## 1.2 Changes to the option.py file

Here we are adding pertinent lines of code to the option.py file in the src folder, to add arguments required for the freeze.py module, the argument save_models_each and an additional line to avoid errors from the argument parser if we want to load the model externally of the script.

In [ ]:
"""
NOTE: This will always try to read the option.py file from a file
named as option-backup.py, which corresponds to the original
option.py from the EDSR-PyTorch repository.

If such file is not found, the code below will assume it's a fresh
clone of the repository, and it will make it.
"""

import os

# 1. Add this line because the argument parser was giving errors on Google Colab
new_line = 'parser.add_argument("-f", "--file", required=False)\n\n'

# 2. Add to the arg parser the arguments for freezing the model before training
freezing_arguments = [
    "# Transfer Learning specifications\n",
    "parser.add_argument('--param_to_freeze', type=str, default='',\n",
    "                    help='named parameters in the model to freeze during training')\n",
    "parser.add_argument('--body_to_freeze', type=str, default='',\n",
    "                    help='range of layers to freeze in the body of the model during training')\n",
    "parser.add_argument('--tail_to_freeze', type=str, default='',\n",
    "                    help='range of layers to freeze in the tail of the model during training')\n",
    "parser.add_argument('--print_frozen_param', action='store_true',\n",
    "                    help='print the number of parameters from the model frozen during training')\n",
    "parser.add_argument('--torchinfo_summary', action='store_true',\n",
    "                    help='print the model summary using torchinfo')\n",
    "parser.add_argument('--torchinfo_inputsize', type=str, default='510,339',\n",
    "                    help='input size of a test image to use for printing torchinfo summary')\n\n"
]
#Note: spaces/tabulations are important.

# 3. Add line to save models each number of epochs instead of every epoch
savemodels_lines = [
    "parser.add_argument('--save_models_each', type=str, default='1',\n",
    "                    help='save models each this number of epochs')\n\n"
]

# File paths
option_file_path = os.path.join(dir_src,'option.py')                 # Original
option_file_path_backup = os.path.join(dir_src,'option-backup.py')   # Back-up

# If the back-up file is there, read it, or make it if not
if os.path.isfile(option_file_path_backup):
  # Read the content of the back-up file
  with open(option_file_path_backup, 'r') as file:
      lines = file.readlines()
else:
  # Read the content of the original file
  with open(option_file_path, 'r') as file:
      lines = file.readlines()
  # Write the content to the back-up file
  with open(option_file_path_backup, 'w') as file:
      file.writelines(lines)

# 1. Insert new_line at the desired location (between lines 145 and 146)
lines.insert(145, new_line)

# 2. Insert the freezing arguments lines at the desired location (line 145)
for idx,line in enumerate(freezing_arguments):
  lines.insert(145+idx, line)

# 3. Insert the save models lines at the desired location (line 144)
for idx,line in enumerate(savemodels_lines):
  lines.insert(144+idx, line)

# Write the modified content back to the file
with open(option_file_path, 'w') as file:
    file.writelines(lines)

print(f"Option file successfully updated in {option_file_path}")

Option file successfully updated in /content/EDSR-PyTorch/src/option.py


## 1.3 Changes to the main.py file

We need to modify the main.py file to include the import of the freeze module and to apply the freeze_model() function to the model before passing it to the Trainer:

In [ ]:
"""
NOTE: This will always try to read the main.py file from a file
named as main-backup.py, which corresponds to the original
main.py from the EDSR-PyTorch repository.

If such file is not found, the code below will assume it's a fresh
clone of the repository, and it will make it.
"""

import os

# Lines to add
new_line1 = "import freeze\n"
new_line2 = "            _model = freeze.freeze_model(_model, args)\n"
#Note: spaces/tabulations are important.


# File paths
main_file_path = os.path.join(dir_src,'main.py')                 # Original
main_file_path_backup = os.path.join(dir_src,'main-backup.py')   # Back-up

# If the back-up file is there, read it, or make it if not
if os.path.isfile(main_file_path_backup):
  # Read the content of the back-up file
  with open(main_file_path_backup, 'r') as file:
      lines = file.readlines()
else:
  # Read the content of the original file
  with open(main_file_path, 'r') as file:
      lines = file.readlines()
  # Write the content to the back-up file
  with open(main_file_path_backup, 'w') as file:
      file.writelines(lines)

# Insert the new lines at the desired location (from bottom to top)
lines.insert(23, new_line2)
lines.insert(8, new_line1)

# Write the modified content back to the file
with open(main_file_path, 'w') as file:
    file.writelines(lines)

print(f"Option file successfully updated in {main_file_path}")

Option file successfully updated in /content/EDSR-PyTorch/src/main.py


## 1.4 Changes to the _ _ init _ _ file in the "model" folder

To be able to use the argument save_models_each, we need to modify the _ _ init _ _ file inside the "model" folder which is in the src folder of the repository:

In [ ]:
"""
NOTE: This will always try to read the __init__.py file from a file
named as init-backup.py, which corresponds to the original
__init__.py from the EDSR-PyTorch repository.

If such file is not found, the code below will assume it's a fresh
clone of the repository, and it will make it.
"""

# Add these lines
new_line1 = '        self.save_models_each = args.save_models_each\n'
new_line2 = '            if (int(epoch) % int(self.save_models_each) == 0):\n'

# Modify these lines (add tab to the original ones)
lines_to_modify = [
    '                save_dirs.append(\n',
    '                    os.path.join(apath, \"model_{}.pt\".format(epoch))\n',
    '                )\n'
]
#Note: spaces/tabulations are important.

# Specify the file path
init_file_path = os.path.join(dir_src,'model', '__init__.py')             # Original
init_file_path_backup = os.path.join(dir_src,'model', 'init-backup.py')   # Back-up

# If the back-up file is there, read it, or make it if not
if os.path.isfile(init_file_path_backup):
  # Read the content of the back-up file
  with open(init_file_path_backup, 'r') as file:
      lines = file.readlines()
else:
  # Read the content of the original file
  with open(init_file_path, 'r') as file:
      lines = file.readlines()
  # Write the content to the back-up file
  with open(init_file_path_backup, 'w') as file:
      file.writelines(lines)


# Modify the lines
for idx,line in enumerate(lines_to_modify):
  lines[73+idx] = line

# Insert lines at the desired locations
lines.insert(73, new_line2)
lines.insert(32, new_line1)

# Write the modified content back to the file
with open(init_file_path, 'w') as file:
    file.writelines(lines)

# 2. Place the datasets

We used two different datasets:

* General dataset (TCIA): The "Custom" folder inside of "image-data-general".
* Dedicated dataset (Humanitas): The "Custom" folder inside of "image-data-dedicated".

We placed both "image-data" folders inside of the "EDSR-PyTorch" repository.

We need to properly provide the path to our dataset in the arguments we will pass to the main.py script. (Argument dir_data).

# 3. Get a pretrained model

We placed our trained models in a "pre-train" folder inside of the "EDSR-PyTorch" repository.

We need to properly provide the path to our pretrained model (state dictionary file in .pt format) in the arguments we will pass to the main.py script. (Argument pre_train).

We can get the pretrained EDSR x4 by the authors with:

In [ ]:
import os
import urllib.request

# Pre-train folder
if not os.path.exists(dir_pretrain):
    os.makedirs(dir_pretrain, exist_ok=True)

# Pretrained model
pretrain_model = "edsr_x4-4f62e9ef.pt"
pretrain_model_path = os.path.join(dir_pretrain, pretrain_model)

# Download it if not present
if not os.path.isfile(pretrain_model_path):
  url = "https://cv.snu.ac.kr/research/EDSR/models/edsr_x4-4f62e9ef.pt"
  with urllib.request.urlopen(url) as response, open(pretrain_model_path, 'wb') as out_file:
    data = response.read()
    out_file.write(data)
  print(f"Pretrained model {pretrain_model} has been downloaded inside {dir_pretrain}")
else :
  print(f"Using pretrained model {pretrain_model_path}")

Pretrained model edsr_x4-4f62e9ef.pt has been downloaded inside /content/EDSR-PyTorch/pre-train


# 4. Setting up a template

We can set a template EDSR_custom to use to pass arguments to the main.script by adding it to the template.py file of the EDSR-PyTorch repository:

In [ ]:
"""
NOTE: This will always try to read the template.py file from a file
named as template-backup.py, which corresponds to the original
template.py from the EDSR-PyTorch repository.

If such file is not found, the code below will assume it's a fresh
clone of the repository, and it will make it.
"""

# Lines to add to the file
template = [
    '    if args.template.find(\'EDSR_custom\') >= 0:\n',
    '        args.dir_data =  \"../image-data\"\n',
    '        args.data_train = \"Custom\"\n',
    '        args.data_test = \"Custom\"\n',
    '        args.data_range = \"1-2400/2401-2500\"\n',
    '        args.ext = \"sep\"\n',
    '        args.scale = \"4\"\n',
    '        args.model = \"EDSR\"\n',
    '        args.pre_train = \"../pre-train/edsr_x4-4f62e9ef.pt\"\n',
    '        args.n_resblocks = 32\n',
    '        args.n_feats = 256\n',
    '        args.res_scale = 0.1\n',
    '        args.test_every = 100\n',
    '        args.epochs = 11\n',
    '        args.batch_size = 16\n',
    '        args.save = \"edsr_x4-train\"\n'
]
#Note: spaces/tabulations are important.

# Specify the file path
template_file_path = os.path.join(dir_src,'template.py')

# Have a back-up of the original template.py
template_file_path_backup = os.path.join(dir_src,'template-backup.py')

# If the back-up file is there, read it, or make it if not
if os.path.isfile(template_file_path_backup):
  # Read the content of the back-up file
  with open(template_file_path_backup, 'r') as file:
      lines = file.readlines()
else:
  # Read the content of the original file
  with open(template_file_path, 'r') as file:
      lines = file.readlines()
  # Write the content to the back-up file
  with open(template_file_path_backup, 'w') as file:
      file.writelines(lines)

# Insert the new lines at the desired location (on line 54)
for idx,line in enumerate(template):
  lines.insert(54+idx, line)

# Write the modified content back to the file
with open(template_file_path, 'w') as file:
    file.writelines(lines)

print(f"Template successfully added to {template_file_path}")

For different trainings, we should remember to properly indicate the arguments dir_data, data_range, pre_train, epochs and save.

# 5. Running main.py on a terminal

## 5.1 For training

With all set, we just need to run main.py on a terminal, by providing our arguments of interest (if not already on the template). Arguments we didn't include on the template are:

* param_to_freeze: List of the layers names we would want to freeze during training, if any.
* body_to_freeze: Range of sub-layers of the "body" layer of the model we would want to freeze, if any.
* tail_to_freeze: Range of sub-layers of the "tail" layer of the model we would want to freeze, if any.
* print_frozen_param = True (if we want to print the number of frozen parameters)
* save_models = True (to save models beside "best" and "latest")
* save_models_each: The number of how many epochs must pass to save a model during training.
* chop = True (memory efficient forward of the model, to avoid getting errors due to insufficient memory)

And we can finally perform a training session by running the command on a terminal:

In [ ]:
# Command to train the model (from the src folder)
!python main.py --template EDSR_custom --save_models --save_models_each 5 --chop

## 5.2 For a quick testing

We prepared a main_use.py script as a simpler alternative of the main.py script. This file must be placed in the src folder of the repository.

We can perform a quick application of a trained model by providing the following arguments:

* dir_demo: Path to the folder containing the images we want to upscale (in either PNG or JPG format).
* pretrain_path: Path to the state dictionary file of the EDSR x4 model we want to use to upscale the images.

And we can run the following command on a terminal:

In [ ]:
# Command to train the model (from the src folder)
!python main_use.py --data_test Demo --dir_demo {dir_demo} --scale 4 --n_resblocks 32 --n_feats 256 --res_scale 0.1 --pre_train {pretrain_path} --test_only --save_results

We then can expect a folder named "experiment" inside the folder dir_demo, containing the Super-Resolution upscalings of the images inside of dir_demo.